<a href="https://colab.research.google.com/github/Oswald-Adzapah/IMAGE-CLASSIFICATION-/blob/main/IMAGE_CLASSIFICATION_ALGORITHM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [191]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import layers, Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
import random as rd

In [192]:
# Directory paths
image_dir = '/content/drive/MyDrive/archive/train'
Timage_dir = '/content/drive/MyDrive/archive/test'

In [ ]:
# Function to create labels
def create_labels(image_dir):
    labels = os.listdir(image_dir)
    label_to_code = {label: code for code, label in enumerate(labels)}
    image_paths, image_labels = [], []

    for label in labels:
        label_code = label_to_code[label]
        label_dir = os.path.join(image_dir, label)

        for image_filename in os.listdir(label_dir):
            image_path = os.path.join(label_dir, image_filename)
            image_paths.append(image_path)
            image_labels.append(label_code)

    return image_paths, image_labels

create_labels(image_dir)

In [194]:
# Create labels for training and test images
image_paths, image_labels = create_labels(image_dir)
tst_image_paths, tst_image_labels = create_labels(Timage_dir)


In [195]:
# Batch size and image size
batch_size = 40
image_size = (180, 180)

In [ ]:
# Create datasets
train_ds = tf.keras.utils.image_dataset_from_directory(
    image_dir,
    batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    Timage_dir, batch_size=batch_size,
    image_size=image_size,
    shuffle=True,
)

class_names = train_ds.class_names

nrows = 4
ncols = 4
plt.figure(figsize=(ncols*4, nrows*4))
for images, labels in train_ds.take(1):
  for i in range(12):
    ax = plt.subplot(4, 4, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("on")

# Preprocess images
def preprocess_images(images, label):
    return images / 255.0, label

train_ds = train_ds.map(preprocess_images)
val_ds = val_ds.map(preprocess_images)

In [197]:
# Define the model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(180, 180, 3)),
    Dropout(0.5),
    MaxPooling2D((2, 2)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(64,
    activation ='relu'),
    Dense(1, activation='sigmoid')
])

In [198]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Fit the model
model.fit(train_ds, epochs=25)

In [200]:
# Evaluate the model
model.evaluate(val_ds)


In [ ]:
# Predict on a random test image
idx = rd.randint(0, len(tst_image_labels) - 1)
tst_image = plt.imread(tst_image_paths[idx])
plt.imshow(tst_image)
plt.show()

tst_image = tf.image.resize(tst_image, (180, 180))  # Resize to match the model input shape
tst_image = tf.expand_dims(tst_image, axis=0)  # Add batch dimension

y_pred = model.predict(tst_image)
print(y_pred)

if y_pred[0, 0] > 0.5:
    pred = 'NORMAL'
else:
    pred = 'COVID'

print('Our Model says it is:', pred)

In [ ]:
## putting all y_pred in a list and making it an array
all_y_pred= []
for idx in range(len(tst_image_labels)):
  tst_image= plt.imread(tst_image_paths[idx])
  tst_image = tf.image.resize(tst_image, (180, 180))
  tst_image = tf.expand_dims(tst_image, axis=0)

  y_pred= model.predict(tst_image)
  all_y_pred.append(y_pred)

all_y_pred_array = np.array(all_y_pred)
print(all_y_pred_array)


In [ ]:
all_y_pred_array = all_y_pred_array.reshape(-1)
all_y_pred_array_binary = (all_y_pred_array > 0.5).astype(int)
tst_image_labels_array = np.array(tst_image_labels)

conf_matrix = confusion_matrix(tst_image_labels_array, all_y_pred_array_binary)
print("Confusion Matrix:")
print(conf_matrix)
